In [1]:
import pandas as pd

In [2]:
# df = pd.read_csv("sentiment_processed/part-00296-57f7270d-f9cc-406c-8b8c-af8163fc3683-c000_processed.csv")
# df['DateTime'] = pd.to_datetime(df['DateTime'])
# df['Date_hourly'] = df['DateTime'].dt.strftime('%Y%m%d%H')
# df.groupby("Date_hourly").first().to_csv("mock_regression_data.csv")

In [72]:
# aggregate function
def get_aggregated_df(df,multiplier,sentiment_list):
    """Obtain normalised weighted average for each sentiment. Weighted by pagerank score of each article and grouped by hourly date"""
    
    new_sentiment_nameList = []
    aggregated_df = pd.DataFrame()
    for sentiment in sentiment_list:
        #removing rows with incorrect datatype
        non_float_df = pd.to_numeric(df[sentiment], errors='coerce').isna()
        df = df[~non_float_df]
        #calculating aggregate value
        weighted_sentiment_name = f"weighted_{sentiment}"
        df[sentiment] = df[sentiment].astype(float)
        df[weighted_sentiment_name] = df[sentiment] * df[multiplier]
        new_sentiment_nameList.append(weighted_sentiment_name)
        avg_weighted_sentiment = df.groupby('Date_hourly')[weighted_sentiment_name].mean().reset_index()
        min_value = avg_weighted_sentiment[weighted_sentiment_name].min()
        max_value = avg_weighted_sentiment[weighted_sentiment_name].max()
        avg_weighted_sentiment[weighted_sentiment_name] = (avg_weighted_sentiment[weighted_sentiment_name] - min_value) / (max_value - min_value)
        avg_weighted_sentiment.set_index("Date_hourly",inplace=True)
        aggregated_df = pd.concat([aggregated_df,avg_weighted_sentiment],axis=1)
    return aggregated_df


# 1. Sample

In [30]:
sample_df = pd.read_csv("sampling/final_sample_processed_with_pageRank.csv",low_memory=False)
sample_df = sample_df[["DocTone","positive_fb","negative_fb","neutral_fb","page_rank_decimal","Date_hourly"]]


In [58]:
sample_df

,DocTone,positive_fb,negative_fb,neutral_fb,page_rank_decimal,Date_hourly
0,3.03823991618648,0.040248,0.022995,0.936758,5.72,2024061402
1,4.48430493273543,0.441696,0.009332,0.548972,5.48,2024061402
2,0.212314225053079,0.418814,0.028759,0.552427,7.63,2024061402
3,-7.09146968139774,0.026196,0.674003,0.299801,5.16,2024061402
4,3.03823991618648,0.043051,0.021397,0.935552,5.72,2024061402
...,...,...,...,...,...,...
196844,-2.2133526850508,0.110620,0.033546,0.855833,4.88,2024040209
196845,0.386847195357834,0.023493,0.032889,0.943618,4.78,2024040209
196846,-0.975177304964539,0.072654,0.136838,0.790508,7.63,2024040209
196847,-5.34722222222222,0.034147,0.814993,0.150860,8.18,2024040209


In [73]:
df = sample_df.copy()
multiplier = "page_rank_decimal"
sentiment_list = ["positive_fb","negative_fb","neutral_fb","DocTone"]

aggregated_df = get_aggregated_df(df,multiplier,sentiment_list)


In [75]:
# aggregated_df.to_csv("final_sample_finbert.csv")

In [76]:
aggregated_df

,weighted_positive_fb,weighted_negative_fb,weighted_neutral_fb,weighted_DocTone
Date_hourly,,,,
2024010100,0.266860,0.730939,0.302638,0.287875
2024010101,0.355989,0.260473,0.568606,0.488518
2024010102,0.259909,0.491577,0.420953,0.384908
2024010103,0.320862,0.308524,0.607849,0.383106
2024010104,0.197884,0.331869,0.630541,0.457622
...,...,...,...,...
2024101716,0.562321,0.393287,0.444060,0.456393
2024101720,0.683561,0.186674,0.367867,0.549744
2024101722,0.294753,0.300548,0.503819,0.465461
